# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath = '../output_data/cities.csv'
f = pd.read_csv(filepath)
df = pd.DataFrame(f)
df = df.drop(columns=['Unnamed: 0'])

df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,yellowknife,62.46,-114.35,18.00,92,75,10.29,CA,1607376601
1,narsaq,60.92,-46.05,21.20,53,40,3.36,GL,1607376586
2,leningradskiy,69.38,178.42,7.68,86,69,3.74,RU,1607376820
3,ushuaia,-54.80,-68.30,50.00,76,75,13.87,AR,1607376491
4,castro,-24.79,-50.01,66.31,93,76,5.44,BR,1607376571


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = df[["Lat", "Lng"]].astype(float)
humidity_map = df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_map, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
idealTemp = df.loc[(df['Max Temp'] > 70) & (df['Max Temp'] < 80)]
idealTemp_df = idealTemp[['City', 'Country', 'Lat', 'Lng', 'Max Temp']]
idealWind = df.loc[df['Wind Speed'] < 10]
idealWind_df = idealWind[['City', 'Lat', 'Lng', 'Wind Speed']]
idealClouds = df.loc[df['Cloudiness'] == 0]
idealCloud_df = idealClouds[['City', 'Lat', 'Lng', 'Cloudiness']]

ideal_TempWind_df = pd.merge(idealTemp_df, idealWind_df, on='City', how='inner')
ideal_wx_df = pd.merge(ideal_TempWind_df, idealCloud_df, on='City')

ideal_wx_df = ideal_wx_df[['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Wind Speed', 'Cloudiness']]

hotel_df = pd.DataFrame(ideal_wx_df)
hotel_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness
0,guerrero negro,MX,27.98,-114.06,76.62,4.36,0
1,ler,SS,8.30,30.14,79.99,5.86,0
2,saint-philippe,RE,-21.36,55.77,75.20,4.70,0
3,keti bandar,PK,24.14,67.45,72.77,6.35,0
4,maceio,BR,-9.67,-35.74,75.20,5.82,0
5,chokwe,MZ,-24.53,32.98,70.56,5.08,0
6,saint-leu,RE,-21.15,55.28,75.20,5.82,0
7,muscat,OM,23.61,58.59,75.20,1.12,0
8,vallenar,CL,-28.57,-70.76,79.70,8.23,0
9,tijara,IN,27.93,76.85,71.01,1.79,0


In [5]:
hotel_df['Hotel Name'] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

target_type = 'hotel'
target_radius = 5000

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    location = f'{lat},{lng}'
    
    response = requests.get(f'{base_url}location={location}&keyword={target_type}&radius={target_radius}&key={g_key}').json()
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except IndexError:
        print("Missing field/result... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = np.nan

Closest hotel is The Halfway Inn.
Missing field/result... skipping.
Closest hotel is Les Embruns Du Baril.
Missing field/result... skipping.
Closest hotel is Jatiúca Hotel & Resort.
Closest hotel is Hotel Sorriso de Caridade.
Closest hotel is Iloha Seaview Hotel.
Closest hotel is Al Bustan Palace, A Ritz-Carlton Hotel.
Closest hotel is Hotel Cecil.
Closest hotel is Tijara Fort-Palace - 19th Century, Alwar.
Closest hotel is Hotel Santa Fé.
Closest hotel is Lamar Resort Abu Soma.
Closest hotel is Hôtel Niara Belly.
Closest hotel is Hotel East Lagoon.
Closest hotel is Le Battant Des Lames.


In [6]:
hotel_df.dropna(axis=0, how='any', inplace=True)

In [7]:
hotel_name = hotel_df['Hotel Name'].tolist()

marker_locations = hotel_df[['Lat','Lng']]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations, 
                             info_box_content=[f'Hotel: {row}' for row in hotel_name])
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Display figure
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, 
                             info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))